### Psycopg2

1. Wczytaj dane z pliku `cars.csv` i wyodrębnij z niego kolumny:
   - `price`
   - `brand`
   - `fuel`
   - `power`
   - `prod_year`


Następnie napisz zapytanie `CREATE`, które utworzy tabelę o takim samym schemacie + dodatkowa kolumna `id` typu `serial`. Wykonaj zapytanie za pomocą `psycopg2`

---
(czas: 16 min.)

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("cars.csv", usecols=["price", "brand", "fuel", "power", "prod_year"]).iloc[:10]
df

,price,brand,fuel,power,prod_year
0,32900,Škoda,Diesel,115.0,2017
1,32500,BMW,Diesel,116.0,2012
2,7900,Fiat,Benzyna,69.0,2012
3,39990,Kia,Diesel,115.0,2012
4,42900,Kia,Diesel,115.0,2012
5,19400,BMW,Benzyna+LPG,177.0,2005
6,106600,Renault,Benzyna,140.0,2020
7,32900,Dacia,Benzyna,102.0,2016
8,32999,Opel,Diesel,163.0,2015
9,36800,Volkswagen,Diesel,140.0,2012


In [50]:
import psycopg2
from psycopg2.extras import RealDictCursor

In [53]:
create_query = """CREATE TABLE cars (
    id SERIAL,
    price INTEGER,
    brand TEXT,
    fuel TEXT,
    power SMALLINT,
    prod_year SMALLINT
)
"""

In [77]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="postgres", host="localhost", cursor_factory=RealDictCursor)
cursor = conn.cursor()

In [60]:
query = "select * from tasks"

In [65]:
cursor.execute(query)

In [66]:
results = cursor.fetchall()

In [69]:
results[0]["description"]

'Task description'

In [83]:
cursor.closed

True

In [79]:
conn

<connection object at 0x7feaa31b3ec0; dsn: 'user=postgres password=xxx dbname=postgres host=localhost', closed: 0>

In [72]:
conn.close()

In [80]:
cursor.close()

In [55]:
cursor.execute(create_query)
conn.commit()

DuplicateTable: relation "cars" already exists


In [8]:
# ...

2. Napisz kod, który na podstawie numeru indeksu wiersza dataframe'a, wyciągnie ten wiersz z tabeli, utworzy zapytanie `INSERT` a następnie wykona je na bazie.

Podpowiedź:

Poniższy kod zamieni typy danych w rekordzie z df-a z numpyowych na zwykłe. Będzie to przydatne podczas insertowania.

`[x.item() if isinstance(x, np.generic) else x for x in record.values]`

---

(czas: 20 min.)

In [19]:
import numpy as np

In [20]:
def get_row_from_df(df, idx):
    return df.loc[idx]

In [36]:
def get_values_from_record(record):
    return [x.item() if isinstance(x, np.generic) else x for x in record.values]

In [29]:
def generate_insert_query_template(record):
    return f"INSERT INTO cars(price, brand, fuel, power, prod_year) VALUES ({','.join(['%s']*len(record))})"

In [32]:
generate_insert_query_template(record)

'INSERT INTO cars(price, brand, fuel, power, prod_year) VALUES (%s,%s,%s,%s,%s)'

In [31]:
record = get_row_from_df(df, 3)

In [33]:
def run_query(query_template, values):
    cursor.execute(query_template, values)
    conn.commit()

In [37]:
for i in range(10):
    record = get_row_from_df(df, i)
    values = get_values_from_record(record)
    query = generate_insert_query_template(record)
    run_query(query, values)

3. Napisz funkcję, która przyjmie zapytanie typu `SELECT` i wykona je na bazie.

In [7]:
# ...

4. Napisz funkcję, która przyjmie `id` wiersza w tabeli, nazwę kolumny oraz nową wartość a następnie zupdatuje tabelę.

In [5]:
# ...

5. Napisz funkcję, która przyjmie `id` wiersza w tabeli a następnie go usunie.

In [6]:
# ...

### SQLAlchemy

1. Utwórz klasę odpowiadającą tabeli `cars` a następnie samą tabelę za pomocą `SQLAlchemy`.


---
(czas: 16 min.)

In [47]:
from sqlalchemy import Column, Integer, Boolean, VARCHAR, String
from sqlalchemy.sql.expression import false
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

In [42]:
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
database = "postgres"

In [43]:
Base = declarative_base()
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

In [44]:
engine = create_engine(connection_string)

In [45]:
Session = sessionmaker(bind=engine)
session = Session()

In [48]:
class CarsTable(Base):
    __tablename__ = "cars"

    id = Column("id", Integer, primary_key=True)
    price = Column("price", Integer)
    brand = Column("brand", String)
    fuel = Column("fuel", String)
    power = Column("power", Integer)
    prod_year = Column("prod_year", Integer)

    def __repr__(self):
        return f"Cars(id={self.id})"

In [49]:
Base.metadata.create_all(bind=engine)

2. Napisz kod, który doda do tabeli wiersz z dataframe'a wykorzystując `SQLAlchemy`.

   
---
(czas: 8 min.)

In [3]:
# ...

3. Napisz kod, który wyciągnie dane z dataframe'a na różne sposoby

In [4]:
# ...